# Signatures

## Syntax & Correctness

:::{note} Definition: Signature Scheme
:label: def-signature-scheme

A signature scheme $\mathsf{Sig}$ is a tuple of polynomial-time algorithms $(\mathsf{Setup}, \mathsf{KeyGen}, \mathsf{Sign}, \mathsf{Verify})$ where:

- $\mathsf{Setup}(1^\lambda) \rightarrow \mathit{pp}$ is a probabilistic algorithm that takes the security parameter as input and outputs public parameters $\mathit{pp}$ including the message space $\mathcal{M}$ and the signature space $\mathcal{S}$.
- $\mathsf{KeyGen}(\mathit{pp}) \rightarrow (\mathit{sk}, \mathit{pk})$ is a probabilistic algorithm that takes public parameters $\mathit{pp}$ as input and outputs a secret key $\mathit{sk}$ and a public key $\mathit{pk}$.
- $\mathsf{Sign}(\mathit{pp}, \mathit{sk}, m) \rightarrow \sigma$ is a probabilistic algorithm that takes public parameters $\mathit{pp}$, a secret key $\mathit{sk}$, and a message $m \in \mathcal{M}$ as input and outputs a signature $\sigma \in \mathcal{S}$.
- $\mathsf{Verify}(\mathit{pp}, \mathit{pk}, m, \sigma) \rightarrow \{0, 1\}$ is a deterministic algorithm that takes public parameters $\mathit{pp}$, a public key $\mathit{pk}$, a message $m \in \mathcal{M}$, and a signature $\sigma \in \mathcal{S}$ as input and outputs $1$ (accept) if the signature is valid, and outputs $0$ (reject) otherwise.

:::

:::{note} Definition: Correctness
:label: def-sig-correctness

A signature scheme $\mathsf{Sig} = (\mathsf{Setup}, \mathsf{KeyGen}, \mathsf{Sign}, \mathsf{Verify})$ is *perfectly correct* if for all $\lambda \in \mathbb{N}$ and all messages $m \in \mathcal{M}$:

$$
\Pr\left[\mathsf{Verify}(\mathit{pp}, \mathit{pk}, m, \sigma) = 1 \;\middle|\; \begin{array}{l}
  \mathit{pp} \leftarrow \mathsf{Setup}(1^\lambda) \\
  (\mathit{sk}, \mathit{pk}) \leftarrow \mathsf{KeyGen}(\mathit{pp}) \\
  \sigma \leftarrow \mathsf{Sign}(\mathit{pp}, \mathit{sk}, m)
\end{array}\right] = 1
$$

:::

In other words, a signature scheme is correct if honestly generated signatures always verify correctly.

## Security

The standard security notion for digital signatures is existential unforgeability under chosen message attack (EUF-CMA). Unlike one-time signatures, the adversary can request signatures on multiple messages.

:::{note} Definition: EUF-CMA Security
:label: def-euf-cma

A signature scheme $\mathsf{Sig}$ is *existentially unforgeable under chosen message attack (EUF-CMA)* if for all PPT adversaries $\mathcal{A}$:

$$
\mathrm{Adv}_{\mathsf{Sig}}^{\text{EUF-CMA}}(\mathcal{A}) \coloneqq \Pr[\mathsf{Game}~\mathsf{EUF\text{-}CMA}_{\mathsf{Sig}} = 1] = \mathsf{negl}
$$

where $\mathsf{Game}~\mathsf{EUF\text{-}CMA}_{\mathsf{Sig}}$ is defined below.

:::

:::{tip} The EUF-CMA security game for digital signatures
:label: fig-euf-cma-sig

**Game $\mathsf{EUF\text{-}CMA}_{\mathsf{Sig}}$:**

```
params ← Setup(1^λ)
(sk, pk) ← KeyGen(params)
Q ← ∅
(m*, σ*) ← A^Sign(params, pk)
return m* ∉ Q ∧ Verify(params, pk, m*, σ*) = 1
```

**Oracle Sign(m):**

```
σ ← Sign(params, sk, m)
Q ← Q ∪ {m}
return σ
```

:::

## Schnorr Signatures

:::{note} Definition: Schnorr Signature Scheme
:label: def-schnorr

Let $\mathsf{GrGen}$ be a group generation algorithm and $\mathsf{H}$ be a hash function with output space $\mathbb{Z}_p$. The Schnorr signature scheme $\mathsf{SchnorrSig}[\mathsf{GrGen}, \mathsf{H}]$ is defined as follows:

- $\mathsf{Setup}(1^\lambda) \rightarrow \mathit{pp}$: Run $(\mathbb{G}, p, g) \leftarrow \mathsf{GrGen}(1^\lambda)$ and $\kappa \leftarrow \mathsf{H.Gen}(1^\lambda)$. The message space is $\mathcal{M} = \{0,1\}^*$ and the signature space is $\mathcal{S} = \mathbb{G} \times \mathbb{Z}_p$. Output $\mathit{pp} = ((\mathbb{G}, p, g), \kappa, \mathcal{M}, \mathcal{S})$.

- $\mathsf{KeyGen}(\mathit{pp}) \rightarrow (\mathit{sk}, \mathit{pk})$: Sample $x \xleftarrow{\$} \mathbb{Z}_p$. Compute $X = g^x$. Output $\mathit{sk} = x$ and $\mathit{pk} = X$.

- $\mathsf{Sign}(\mathit{pp}, \mathit{sk}, m) \rightarrow \sigma$: Parse $\mathit{pp} = ((\mathbb{G}, p, g), \kappa, \mathcal{M}, \mathcal{S})$ and $\mathit{sk} = x$. Sample $r \xleftarrow{\$} \mathbb{Z}_p$. Compute:
  $$
  \begin{align*}
  R &= g^r \\
  c &= \mathsf{H.Eval}(\kappa, (R, m)) \\
  s &= r + c \cdot x \bmod p
  \end{align*}
  $$
  Output $\sigma = (R, s)$.

- $\mathsf{Verify}(\mathit{pp}, \mathit{pk}, m, \sigma) \rightarrow \{0, 1\}$: Parse $\mathit{pp} = ((\mathbb{G}, p, g), \kappa, \mathcal{M}, \mathcal{S})$, $\mathit{pk} = X$ and $\sigma = (R, s)$. Compute $c = \mathsf{H.Eval}(\kappa, (R, m))$. Accept if and only if:
  $$
  g^s = R \cdot X^c
  $$

:::

:::{important} Lemma: Correctness of Schnorr Signatures
:label: lem-schnorr-correctness

The Schnorr signature scheme $\mathsf{SchnorrSig}[\mathsf{GrGen}, \mathsf{H}]$ is perfectly correct.

:::

:::{dropdown} **Proof**

Let $\mathit{pp} \leftarrow \mathsf{Setup}(1^\lambda)$ and $(\mathit{sk}, \mathit{pk}) \leftarrow \mathsf{KeyGen}(\mathit{pp})$ where $\mathit{sk} = x$ and $\mathit{pk} = X = g^x$.

Let $\sigma = (R, s) \leftarrow \mathsf{Sign}(\mathit{pp}, \mathit{sk}, m)$.

By the signing algorithm, we have $R = g^r$ for some $r \xleftarrow{\$} \mathbb{Z}_p$, $c = \mathsf{H.Eval}(\kappa, (R, m))$, and $s = r + c \cdot x \bmod p$.

The verification equation holds because:

$$
g^s = g^{r + cx} = g^r \cdot g^{cx} = R \cdot (g^x)^c = R \cdot X^c
$$

Therefore, $\mathsf{Verify}(\mathit{pp}, \mathit{pk}, m, \sigma) = 1$ for all honestly generated signatures.

:::

:::{attention} Remark

In the security analysis that follows, we work in the Random Oracle Model where the hash function $\mathsf{H.Eval}(\kappa, \cdot)$ is modeled as a truly random function.

In the ROM analysis, we write $\mathsf{H}(\cdot)$ instead of $\mathsf{H.Eval}(\kappa, \cdot)$, with the understanding that $\mathsf{H}$ represents a random oracle.

:::

:::{tip} Security reduction for Schnorr signatures in the random oracle model
:label: fig-schnorr-reduction

The figure below shows the EUF-CMA game (left), algorithm $\mathcal{B}$ (middle), and algorithm $\mathcal{C}$ (right). Changes from the previous game are marked.

**Game $\mathsf{EUF\text{-}CMA}_{\mathsf{SchnorrSig}}$:**

```
(G, p, g) ← GrGen(1^λ)
x ←$ Z_p
X := g^x
params := ((G, p, g), {0,1}*, G × Z_p)
T := ∅
Q := ∅
(m*, σ*) ← A^Sign,H_UF(params, X)
(R*, s*) := σ*
c* := H_UF(R*, m*)
return m* ∉ Q ∧ g^s* = R* · X^c*
```

**Oracle Sign(m):**

```
r ←$ Z_p
R := g^r
c := H_UF(R, m)
s := r + c · x mod p
Q := Q ∪ {m}
return (R, s)
```

**Oracle H_UF(y):**

```
if T[y] = ⊥ then
    T[y] ←$ Z_p
return T[y]
```

**Algorithm $\mathcal{B}(\text{gparam}, X; \rho)$:**

```
[params := (gparam, {0,1}*, G × Z_p)]  // changed
T := ∅
Q := ∅
(m*, σ*) ← A^Sign_B,H_B(params, X; ρ)
(R*, s*) := σ*
c* := H_B((R*, m*))
[assert m* ∉ Q]  // changed
[assert g^s* = R* · X^c*]  // changed
[return ((R*, m*), s*)]  // changed
```

**Oracle Sign_B(m):**

```
[s ←$ Z_p]  // changed
[c ←$ Z_p]  // changed
[R := g^s · X^(-c)]  // changed
[assert T[R, m] = ⊥]  // changed
[T[R, m] := c]  // changed
Q := Q ∪ {m}
return (R, s)
```

**Oracle H_B(y):**

```
if T[y] = ⊥ then
    T[y] ←$ Z_p
return T[y]
```

**Algorithm $\mathcal{C}^{[\mathcal{H}]}(\text{gparam}, X; \rho)$:**

```
params := (gparam, {0,1}*, G × Z_p)
T := ∅
Q := ∅
(m*, σ*) ← A^Sign_C,H_C(params, X; ρ)
(R*, s*) := σ*
c* := H_C((R*, m*))
assert m* ∉ Q
assert g^s* = R* · X^c*
return ((R*, m*), s*)
```

**Oracle Sign_C(m):**

```
s ←$ Z_p
c ←$ Z_p
R := g^s · X^(-c)
assert T[R, m] = ⊥
T[R, m] := c
Q := Q ∪ {m}
return (R, s)
```

**Oracle H_C(y):**

```
if T[y] = ⊥ then
    [T[y] ← H(y)]  // changed
return T[y]
```

:::

:::{important} Lemma
:label: lem-schnorr-to-single

Let $\mathcal{A}$ be an adversary against the EUF-CMA security of $\mathsf{SchnorrSig}[\mathsf{GrGen}, \mathsf{H}]$ in the random oracle model for $\mathsf{H}$ making at most $q_s$ queries to $\mathsf{Sign}$ and at most $q_h$ queries to $\mathsf{H}$.

Let $\mathsf{InpGen}$ be the algorithm which on input $1^\lambda$ runs $(\mathbb{G}, p, g) \leftarrow \mathsf{GrGen}(1^\lambda)$, draws $X \xleftarrow{\$} \mathbb{G}$, and returns $((\mathbb{G}, p, g), X)$.

Consider algorithm $\mathcal{C}$ defined above that takes as input $(\text{gparam}, X) \leftarrow \mathsf{InpGen}(1^\lambda)$, has access to a random oracle $\mathsf{H}$, and runs adversary $\mathcal{A}$. Then $\mathcal{C}$ makes at most $q_h + 1$ queries to $\mathsf{H}$, and satisfies

$$
\left|\mathrm{Adv}_{\mathsf{InpGen}}^{\text{Single}}(\mathcal{C}) - \mathrm{Adv}_{\mathsf{SchnorrSig}[\mathsf{GrGen}, \mathsf{H}]}^{\text{EUF-CMA}}(\mathcal{A})\right| \leq \frac{q_s(q_s + q_h)}{2^{\lambda-1}}
$$

with $\mathrm{Adv}_{\mathsf{InpGen}}^{\text{Single}}(\mathcal{C})$ as defined in the forking lemma.

Moreover, when $\mathcal{C}$ succeeds (returns $(z, \mathit{aux})$ for some $(z, \mathit{aux})$), then $z = (R^*, m^*)$, $\mathit{aux} = s^*$, and

$$
g^{s^*} = R^* \cdot X^{\mathsf{H}(R^*, m^*)}
$$

:::

The proof is left as an exercise (see Exercise 3).

:::{important} Theorem: Schnorr signatures are EUF-CMA secure
:label: thm-schnorr-euf-cma

Let $\mathsf{GrGen}$ be a group generation algorithm for which the discrete logarithm problem is hard and let $\mathsf{H}$ be a hash function with output space $\mathbb{Z}_p$.

Then the Schnorr signature scheme $\mathsf{SchnorrSig}[\mathsf{GrGen}, \mathsf{H}]$ is EUF-CMA secure in the random oracle model for $\mathsf{H}$.

More precisely, for any PPT adversary $\mathcal{A}$ against the EUF-CMA security of $\mathsf{SchnorrSig}[\mathsf{GrGen}, \mathsf{H}]$ making at most $q_s$ queries to the signing oracle and at most $q_h$ queries to the random oracle $\mathsf{H}$, there exists a PPT adversary $\mathcal{D}$ against the discrete logarithm problem such that

$$
\mathrm{Adv}_{\mathsf{SchnorrSig}[\mathsf{GrGen}, \mathsf{H}]}^{\text{EUF-CMA}}(\mathcal{A}) \leq \frac{q_s(q_s + q_h)}{2^{\lambda-1}} + \sqrt{(q_h + 1) \left(\mathrm{Adv}_{\mathsf{GrGen}}^{\text{DLog}}(\mathcal{D}) + \frac{1}{2^{\lambda-1}}\right)}.
$$

:::

The proof is left as an exercise (see Exercise 4).

## Exercises

:::{exercise}
:label: ex-strong-euf-cma

The EUF-CMA security definition above only requires the forged message to be new. An alternative notion called *strong EUF-CMA* would allow the adversary to win if they produce a different signature for a message even if it was previously queried. Define a strong EUF-CMA security game that captures this intuition.

:::

:::{exercise}
:label: ex-schnorr-security-concrete

In the Generic Group Model (GGM), the advantage of solving the discrete logarithm problem with $q$ group operations is bounded by $\mathrm{Adv}_{\mathsf{GrGen}}^{\text{DLog}}(\mathcal{A}) \leq \frac{q^2}{2p}$ where $p$ is the prime group order.

Consider Schnorr signatures over a 256-bit elliptic curve group (e.g., secp256k1 where $p > 2^{255}$). Suppose an adversary can make:

- $q_s = 2^{10}$ signing queries
- $q_h = 2^{85}$ hash queries
- $q = 2^{85}$ group operations

Calculate the concrete EUF-CMA advantage using the bound from the theorem above with $\lambda = 256$, and compare it to the discrete logarithm advantage $\mathrm{Adv}_{\mathsf{GrGen}}^{\text{DLog}}(\mathcal{A})$ in the GGM.

:::

:::{exercise}
:label: ex-schnorr-to-single

Prove the lemma above using the games shown in the figure.

**Hint:** Relate $\mathsf{Game}~\mathsf{EUF\text{-}CMA}_{\mathsf{SchnorrSig}}$ and $\mathsf{Game}~\mathsf{Single}_{\mathsf{InpGen}}[\mathcal{B}]$, then $\mathsf{Game}~\mathsf{Single}_{\mathsf{InpGen}}[\mathcal{B}]$ and $\mathsf{Game}~\mathsf{Single}_{\mathsf{InpGen}}[\mathcal{C}]$.

:::

:::{exercise}
:label: ex-schnorr-euf-cma

Prove the theorem above. Use the lemma and the forking lemma.

:::

:::{exercise}
:label: ex-adaptor-signatures

**Optional:** Study the formalization of adaptor signatures in Gerhart, Schröder, Soni, and Thyagarajan (Eurocrypt 2024). Explain what gaps existed in previous formalizations and how this work addresses them.

:::